### read pickled data

In [1]:
import pandas as pd
import geopandas as gpd

tot_gdf = pd.read_pickle('all_data.pkl')
# tot_gdf

In [2]:
tot_gdf = tot_gdf.fillna(0)

In [3]:
tot_ny = tot_gdf[tot_gdf['state'] == '36']
tot_nyc = tot_ny[tot_ny['county_name'].isin(['Kings County', 'Queens County', 'Bronx County', 'New York County', 'Richmond County'])]
# tot_nyc

In [4]:
tot_bx = tot_ny[tot_ny['county_name'] == 'Bronx County']

### load bokeh packages

In [5]:
from bokeh.models import (
    CDSView,
    ColorBar,
    CustomJS,
    GeoJSONDataSource,
    HoverTool,
    LinearColorMapper,
)
from bokeh.models.widgets import Slider, Dropdown
from bokeh.palettes import brewer
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.io import show, output_notebook, output_file
from bokeh.resources import CDN

# output_notebook()

In [6]:
PLOT_WIDTH = 1200
PLOT_HEIGHT = 1200

# Define color palettes
palette = brewer['BuGn'][8][::-1] # reverse order of colors so higher values have darker colors
# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 300, high = 3000)

### visualize the prediction results with changing factors in Bronx

In [7]:
feature_name = 'prox_markets'
source = GeoJSONDataSource(geojson = tot_bx.to_json())

# Define custom tick labels for color bar.
tick_labels = {
    '0': '0',
    '1000':'1,000',
    '1250':'1,250',
    '1500':'1,500',
    '1750':'1,750',
    '2000':'2,000', 
    '2250':'2,250',
    '2500':'2,500',
    '2750':'2,750',
    '3000':'3,000', 
    '3250':'3,250+'
}
# Create color bar.
color_bar = ColorBar(
    color_mapper = color_mapper, 
    label_standoff = 8,
    width = PLOT_WIDTH,
    height = 20,
    border_line_color = None,
    location = (0,0), 
    orientation = 'horizontal',
    major_label_overrides = tick_labels
)

# Create figure object.
p = figure(
    title = 'Predicted Median Rent in Bronx County by Number of Markets within a Half-Mile', 
    plot_height = PLOT_HEIGHT,
    plot_width = PLOT_WIDTH, 
    toolbar_location = 'below',
    tools = 'pan, wheel_zoom, box_zoom, reset'
)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
# Add patch renderer to figure.
blockgroups = p.patches(
    'xs',
    'ys',
    source=source,
    fill_color={'field' :'med_rent', 'transform' : color_mapper},
    line_color='gray', 
    line_width=0.25, 
    fill_alpha=1
)
# Create hover tool
p.add_tools(
    HoverTool(
        renderers = [blockgroups],
        tooltips = [('NeighborID','@GEOID'), ('Median Rent', '@med_rent'), 
                    ('Predicted Rent', '@prox_markets_2'), 
                    ('Number of Markets', '@prox_markets')]
    )
)
p.add_layout(color_bar, 'below')

# slider
slider = Slider(title='Select a degree of changes for number of markets: ', start=0, end=10, value=0, step=2)

# callback
callback_code = """
    const degree = this.value
    var feature_name_with_degree
    if (degree == 0) {
        feature_name_with_degree = 'y_pred'
    } else {
        feature_name_with_degree = feature_name + '_' + degree
    }
    blockgroups.glyph.fill_color['field'] = feature_name_with_degree;
    console.log(feature_name_with_degree)
    source.change.emit();
"""
callback = CustomJS(
    args=dict(source=source, blockgroups=blockgroups, feature_name=feature_name),
    code=callback_code
)
slider.js_on_change('value', callback)

layout = column(p, slider)

show(layout)
# output_file("rent-vis-2.html", title="Vis 2")